<a href="https://colab.research.google.com/github/hyunjung28/cakd5_lecture/blob/main/DL_predictiong_house_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 보스턴 주택 가격 데이터셋

1970년 중반 보스턴 외곽 지역의 범죄율, 지방세율 등의 데이터가 주어졌을 때 주택 가격의 중간 값을 예측해 보겠습니다.

여기서 사용할 데이터셋은 이전 두 개의 예제와 다릅니다. 데이터 포인트가 506개로 비교적 개수가 적고 404개는 훈련 샘플로 102개는 테스트 샘플로 나누어져 있습니다. 입력 데이터에 있는 각 특성(예를 들어 범죄율)은 스케일이 서로 다릅니다. 어떤 값은 0과 1 사이의 비율을 나타내고 어떤 것은 1과 12 사이의 값을 가지거나 1과 100 사이의 값을 가집니다.


In [ ]:
from tensorflow.keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

65536/57026 [==================================] - 0s 0us/step


In [ ]:
print(train_data.shape)
test_data.shape

(404, 13)


(102, 13)

In [ ]:
#데이터 준비
#테스트 데이터를 정규화 
mean = train_data.mean(axis=0)
train_data -= mean 
std = train_data.std(axis=0)
train_data/=std

test_data -= mean
test_data /= std

In [ ]:
train_data

array([[-0.27224633, -0.48361547, -0.43576161, ...,  1.14850044,
         0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, ..., -1.71818909,
         0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , ...,  0.78447637,
         0.22061726, -1.30850006],
       ...,
       [-0.40202987,  0.99079651, -0.7415148 , ..., -0.71712291,
         0.07943894, -0.67776904],
       [-0.17292018, -0.48361547,  1.24588095, ..., -1.71818909,
        -0.98764362,  0.42083466],
       [-0.40422614,  2.04394792, -1.20161456, ..., -1.30866202,
         0.23317118, -1.15392266]])

In [ ]:
#모델 구성
from tensorflow.keras import models,layers

def bulid_model():
  model = models.Sequential()
  model.add(layers.Dense(64,activation='relu',input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64,activation='relu'))
  #주택 가격을 에측하는 거니깐 활성화함수 안 주고 그냥 바로 출력하게끔 
  model.add(layers.Dense(1))
  model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
  return model

In [ ]:
import numpy as np

k=4
# 몫 구하기
num_val_samples = len(train_data)// k
num_epochs=100
all_scores= []
for i in range(k):
  #검증 데이터 준비 : K번째 분할 
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_target = train_targets[i*num_val_samples: (i+1) * num_val_samples]
    #훈련 데이터 준비 : 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i+1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i+1) * num_val_samples:]],
        axis=0)
    #케라스 모델 구성 
    model = bulid_model()
    model.fit(partial_train_data,partial_train_targets,
              epochs = num_epochs,batch_size=1,verbose=0)
    # 검증 세트로 모델 평가
    val_mse,val_mae = model.evaluate(val_data,val_target,verbose=0)
    all_scores.append(val_mae)

In [ ]:
all_scores

[2.3678834438323975, 2.6588635444641113, 2.5248329639434814, 2.661848545074463]

In [ ]:
np.mean(all_scores)

2.5533571243286133

In [ ]:
k=4
num_epochs=500
all_mae_histories= []
for i in range(k):
  #검증 데이터 준비 : K번째 분할 
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_target = train_targets[i*num_val_samples: (i+1) * num_val_samples]
    #훈련 데이터 준비 : 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i+1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i+1) * num_val_samples:]],
        axis=0)
    #케라스 모델 구성 
    model = bulid_model()
    history = model.fit(partial_train_data,partial_train_targets,
             validation_data = (val_data,val_target),
             epochs = num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mae']

    # 검증 세트로 모델 평가
    all_mae_histories.append(mae_history)

In [ ]:
history.history.keys()

dict_keys(['loss', 'mae', 'val_loss', 'val_mae'])

In [ ]:
average_mae_history = [
                       np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]
                      

In [ ]:
all_mae_histories

[[4.129065990447998,
  3.4990925788879395,
  2.7592670917510986,
  2.69804310798645,
  2.5877275466918945,
  2.3999359607696533,
  2.556945562362671,
  2.641108989715576,
  2.4629294872283936,
  2.0986363887786865,
  2.3364100456237793,
  2.3002185821533203,
  2.070342779159546,
  2.0855085849761963,
  1.9204989671707153,
  2.173485517501831,
  2.252037525177002,
  2.15991473197937,
  1.8995351791381836,
  2.1969497203826904,
  2.071399688720703,
  1.929776906967163,
  2.057154893875122,
  2.0249528884887695,
  1.9359068870544434,
  1.8432931900024414,
  1.9592869281768799,
  2.16085147857666,
  2.3152530193328857,
  1.808098554611206,
  2.2776408195495605,
  2.0916645526885986,
  2.1894824504852295,
  1.9388189315795898,
  1.8600916862487793,
  1.8389155864715576,
  1.8584198951721191,
  1.8432776927947998,
  2.1248621940612793,
  2.0215253829956055,
  2.079658031463623,
  1.9324932098388672,
  2.2655715942382812,
  1.9396132230758667,
  1.8659658432006836,
  2.2317702770233154,
  1.8

In [ ]:
np.mean(all_mae_histories)

2.7033033913969993

In [ ]:
k=4
num_epochs=500
all_mae_histories= []
for i in range(k):
  #검증 데이터 준비 : K번째 분할 
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_target = train_targets[i*num_val_samples: (i+1) * num_val_samples]
    #훈련 데이터 준비 : 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
        train_data[(i+1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
        train_targets[(i+1) * num_val_samples:]],
        axis=0)
    #케라스 모델 구성 
    model = bulid_model()
    history = model.fit(partial_train_data,partial_train_targets,
             validation_data = (val_data,val_target),
             epochs = num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mae']

    # 검증 세트로 모델 평가
    all_mae_histories.append(mae_history)